In [1]:
import pandas

In [2]:
import sklearn

In [3]:
from sklearn import cluster

In [4]:
from sklearn import tree

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
%matplotlib inline

In [7]:
BOSTON_DATA = "Food_Establishment_Inspections_(converted).csv"

In [8]:
CHICAGO_DATA = "Food_Inspections_(converted).csv"

In [9]:
boston_df = pandas.read_csv(BOSTON_DATA)

In [10]:
#boston_violations = boston_df.groupby('violation')

In [11]:
# plot barchart of violation occurrences by full violoation code
#boston_violations.size().plot(kind='bar')

In [12]:
#boston_businesses = boston_df.groupby(['address'])

In [13]:
# plot number of pass/fail
#boston_df.groupby('result').size().plot(kind='bar')

In [14]:
# plot number of pass/fail
#boston_df.groupby('risk').size().plot(kind='bar')

In [15]:
# plot number of pass/fail
#boston_df.groupby(['result', 'risk']).size().plot(kind='bar')

In [16]:
from sklearn import feature_extraction

In [17]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
# map result: Fail to 0 and result: Pass to 1
target = boston_df['result'].map(lambda x: 0 if x == "Fail" else 1)

In [20]:
# extract a subset of the data for machine learning
boston_subset = boston_df[["businessName", "violation", "risk", "city", "result"]]

In [21]:
# some city values are 'NaN', replace them with a blank space
boston_subset = boston_subset.replace(pandas.np.nan,' ', regex=True)

In [22]:
# for performance purpose use only a random sample of 100k records
boston_subset2 = boston_subset.sample(100000)

In [23]:
# split the records into 80k for training and 20k for testing
boston_train, boston_test = train_test_split(boston_subset2, test_size = 0.2)

In [24]:
# map result: Fail to 0 and result: Pass to 1
boston_target = boston_train['result'].map(lambda x: 0 if x == "Fail" else 1)

In [25]:
# remove result column fron boston training data
boston_train = boston_train.drop('result', 1)

In [26]:
vec = DictVectorizer()

In [27]:
boston_model = DecisionTreeClassifier()

In [28]:
# generate a list of dicts of boston inspection data
boston_train_dict = boston_train.to_dict('records')

In [29]:
boston_train_dict[0]

{'businessName': 'Tu Metapan Restaurant',
 'city': 'East Boston',
 'risk': 'Low',
 'violation': '25-4-904.11'}

In [30]:
# extract features based on unique business names, city names, violation codes, and risk
feature_matrix = vec.fit_transform(boston_train_dict).toarray()

In [31]:
# fit a decision tree to the boston inspection data
%time boston_model.fit(feature_matrix, boston_target)

CPU times: user 50min 16s, sys: 8.63 s, total: 50min 24s
Wall time: 50min 37s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [32]:
boston_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [34]:
from sklearn import tree

# max_depth required because a recursion exception thrown if trying to generate full tree
with open("boston_dt2.dot", 'w') as f:
    f = tree.export_graphviz(boston_model, out_file=f, max_depth=10)

In [ ]:
# TODO now to evaluate model with test data